# Research Start

In [1]:
from datetime import datetime
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
import os
import statistics

%matplotlib inline

### Creating Futures Time Series

In [2]:
contract_loc = '/home/nikki/data-sc'
contract_files = list(file for file in os.listdir(contract_loc) if file[-3:] == 'csv')

In [3]:
"""
Current portfolio as of October 2020
8 instruments
Data currently uses continuous non-adjusted futures provided free from Quandle

TODO:
Look for ways to use back-adjusted futures data
"""
# contracts = ['NQZ20-CME', 'UBZ20-CBOT', 'CHRIS/CME_BP', 'CHRIS/CME_GC', 'CHRIS/EUREX_FDAX', 'CHRIS/EUREX_FGBL', 'CHRIS/CME_AD', 'CHRIS/CME_LN']
contracts = ['NQZ20-CME', 'UBZ20-CBOT', 'ZCZ20-CBOT', 'HEZ20-CME', 'GCZ20-COMEX', '6BZ20-CME', '6AZ20-CME', 'FDAXZ20-EUREX']

In [4]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
Run this every weekday at 13:00 GMT+7
"""
data_futures = {}
for contract in contracts:
    data_futures[contract] = pd.read_csv(f'{contract_loc}/{contract}-BarStudyData.csv', usecols=[0, 5], index_col=0, parse_dates=True)

In [5]:
for key, item in data_futures.items():
    print(key, item.index[-1])

NQZ20-CME 2020-11-05 00:00:00
UBZ20-CBOT 2020-11-05 00:00:00
ZCZ20-CBOT 2020-11-04 00:00:00
HEZ20-CME 2020-11-04 00:00:00
GCZ20-COMEX 2020-11-05 00:00:00
6BZ20-CME 2020-11-05 00:00:00
6AZ20-CME 2020-11-05 00:00:00
FDAXZ20-EUREX 2020-11-04 00:00:00


In [8]:
data_futures['FDAXZ20-EUREX'].iloc[-2:]

,Last
7568,12305.0
7569,12559.0


In [7]:
data_futures['FDAXZ20-EUREX'] = data_futures['FDAXZ20-EUREX'].append({
    ' Last':12559.0}, ignore_index=True)

In [14]:
data_futures['FGBLZ20-EUREX'].iloc[-2:]

,Last
7535,176.11
7536,175.81


In [10]:
data_futures['FGBLZ20-EUREX'] = data_futures['FGBLZ20-EUREX'].append({
    ' Last':175.81}, ignore_index=True)

In [11]:
data_futures['HEZ20-CME'].iloc[-2:]

,Last
7970,66.350
7971,67.425


In [10]:
data_futures['HEZ20-CME'] = data_futures['HEZ20-CME'].append({
    ' Last':67.425}, ignore_index=True)

In [14]:
data_futures['ZCZ20-CBOT'].iloc[-2:]

,Last
8019,405.25
8020,409.25


In [13]:
data_futures['ZCZ20-CBOT'] = data_futures['ZCZ20-CBOT'].append({
    ' Last':409.25}, ignore_index=True)

## Rules Logic

In [15]:
def volatility(ts, lookback=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(lookback).std().iloc[-1]

    # alternative caltulation using ewma
#     return ts.pct_change().ewm(span=lookback).std().iloc[-1]

In [16]:
mac_scaling = {2: 180.8,
               4: 124.32,
               8: 83.84,
               16: 57.12,
               32: 38.24,
               64: 25.28}

ewmac_scaling = {2: 10.6,
                 4: 7.5,
                 8: 5.3,
                 16: 3.75,
                 32: 2.65,
                 64: 1.87}

breakout_scaling = {10: 28.6,
                    20: 31.6,
                    40: 32.7,
                    80: 33.5,
                    160: 33.5,
                    320: 33.5}

vol_lookback = 24

forecast_table = pd.DataFrame(columns=['contract', 'rule', 'forecast', 'ins_risk_pct'])

for contract, timeseries_full in data_futures.items():
    timeseries = timeseries_full[' Last']
    ins_risk_pct = volatility(timeseries, vol_lookback) * 16
    ins_risk_pts = ins_risk_pct * timeseries.iloc[-1]
        
    # MAC forecasts
    for lookback, scaling in mac_scaling.items():
        lookback_fast = lookback
        lookback_slow = 4 * lookback_fast
        ma_fast = timeseries.rolling(lookback_fast).mean().iloc[-1]
        ma_slow = timeseries.rolling(lookback_slow).mean().iloc[-1]
        
        mac_raw = ma_fast - ma_slow
        mac_risk = mac_raw / ins_risk_pts
        mac_scaled = min(20, max(mac_risk * scaling, -20))
        
        forecast_table = forecast_table.append({'contract': contract,
                                                'rule': f'mac_{lookback_fast}_{lookback_slow}',
                                                'forecast': mac_scaled,
                                                'ins_risk_pct': ins_risk_pct},
                                               ignore_index=True)
        
    # EWMAC forecasts    
#     for lookback, scaling in ewmac_scaling.items():
#         lookback_fast = lookback
#         lookback_slow = 4 * lookback_fast
#         ewma_fast = timeseries.ewm(span=lookback_fast).mean().iloc[-1]
#         ewma_slow = timeseries.ewm(span=lookback_slow).mean().iloc[-1]
        
#         ewmac_raw = ewma_fast - ewma_slow
#         ewmac_risk = ewmac_raw / ins_risk_pts
#         ewmac_scaled = min(20, max(ewmac_risk * scaling, -20))
        
#         forecast_table = forecast_table.append({'contract': contract,
#                                                 'rule': f'ewmac_{lookback_fast}_{lookback_slow}',
#                                                 'forecast': ewmac_scaled,
#                                                 'ins_risk_pct': ins_risk_pct},
#                                                ignore_index=True)
        
    # Breakout forecasts
    for lookback, scaling in breakout_scaling.items():
        rolling_max = timeseries.rolling(lookback).max().iloc[-1]
        rolling_min = timeseries.rolling(lookback).min().iloc[-1]
        rolling_avg = (rolling_max + rolling_min) / 2
        breakout_raw = (timeseries.iloc[-1] - rolling_avg) / (rolling_max - rolling_min)
        breakout_scaled = min(20, max(breakout_raw * scaling, -20))
        
        forecast_table = forecast_table.append({'contract': contract,
                                                'rule': f'breakout_{lookback}',
                                                'forecast': breakout_scaled,
                                                'ins_risk_pct': ins_risk_pct},
                                               ignore_index=True)

In [17]:
forecast_table.to_clipboard()

In [85]:
data_futures['NQZ20-CME']['stdev'] = data_futures['NQZ20-CME'][' Last'].diff().ewm(span=36).std()
data_futures['NQZ20-CME']['ma_2'] = data_futures['NQZ20-CME'][' Last'].rolling(2).mean()
data_futures['NQZ20-CME']['ma_8'] = data_futures['NQZ20-CME'][' Last'].rolling(8).mean()

data_futures['NQZ20-CME']['mac_2_8'] = data_futures['NQZ20-CME']['ma_2'] - data_futures['NQZ20-CME']['ma_8']
data_futures['NQZ20-CME']['mac_2_8_forecast'] = data_futures['NQZ20-CME']['mac_2_8'] / data_futures['NQZ20-CME']['stdev']
data_futures['NQZ20-CME']['mac_2_8_forecast_abs'] = data_futures['NQZ20-CME']['mac_2_8_forecast'].abs()
data_futures['NQZ20-CME']['mac_2_8_std'] = data_futures['NQZ20-CME']['mac_2_8_forecast'].std()
data_futures['NQZ20-CME']['mac_2_8_scalar'] = 10 / data_futures['NQZ20-CME']['mac_2_8_std']

data_futures['NQZ20-CME']['ewma_2'] = data_futures['NQZ20-CME'][' Last'].ewm(span=2).mean()
data_futures['NQZ20-CME']['ewma_8'] = data_futures['NQZ20-CME'][' Last'].ewm(span=8).mean()
data_futures['NQZ20-CME']['ewmac_2_8'] = data_futures['NQZ20-CME']['ewma_2'] - data_futures['NQZ20-CME']['ewma_8']
data_futures['NQZ20-CME']['ewmac_2_8_forecast'] = data_futures['NQZ20-CME']['ewmac_2_8'] / data_futures['NQZ20-CME']['stdev']
data_futures['NQZ20-CME']['ewmac_2_8_forecast_abs'] = data_futures['NQZ20-CME']['ewmac_2_8_forecast'].abs()
data_futures['NQZ20-CME']['ewmac_2_8_std'] = data_futures['NQZ20-CME']['ewmac_2_8_forecast'].std()

In [86]:
data_futures['NQZ20-CME'].tail(757)

,Last,stdev,ma_2,ma_8,mac_2_8,mac_2_8_forecast,mac_2_8_forecast_abs,mac_2_8_std,ewma_2,ewma_8,ewmac_2_8,ewmac_2_8_forecast,ewmac_2_8_forecast_abs,ewmac_2_8_std,mac_2_8_scalar
Date,,,,,,,,,,,,,,,
2017-10-13,6317.50,32.736572,6305.000,6282.84375,22.15625,0.676804,0.676804,1.199256,6309.350381,6278.602250,30.748130,0.939259,0.939259,0.937484,8.338505
2017-10-16,6338.00,31.995545,6327.750,6297.37500,30.37500,0.949351,0.949351,1.199256,6328.450127,6291.801750,36.648377,1.145421,1.145421,0.937484,8.338505
2017-10-17,6341.50,31.132184,6339.750,6304.62500,35.12500,1.128254,1.128254,1.199256,6337.150042,6302.845806,34.304236,1.101890,1.101890,0.937484,8.338505
2017-10-18,6337.00,30.399094,6339.250,6311.50000,27.75000,0.912856,0.912856,1.199256,6337.050014,6310.435627,26.614387,0.875499,0.875499,0.937484,8.338505
2017-10-19,6315.50,30.261730,6326.250,6315.65625,10.59375,0.350071,0.350071,1.199256,6322.683338,6311.561043,11.122295,0.367537,0.367537,0.937484,8.338505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-09,11724.75,207.888098,11631.875,11461.50000,170.37500,0.819551,0.819551,1.199256,11649.718994,11467.275891,182.443103,0.877602,0.877602,0.937484,8.338505
2020-10-12,12098.25,217.502629,11911.500,11547.87500,363.62500,1.671819,1.671819,1.199256,11948.739665,11607.492360,341.247305,1.568934,1.568934,0.937484,8.338505
2020-10-13,12094.00,211.818071,12096.125,11612.78125,483.34375,2.281882,2.281882,1.199256,12045.579888,11715.605169,329.974719,1.557821,1.557821,0.937484,8.338505


In [77]:
data_futures['NQZ20-CME'][data_futures['NQZ20-CME']['ewmac_2_8_forecast'] < 0]

,Last,stdev,ma_2,ma_8,mac_2_8,ewma_2,ewma_8,ewmac_2_8,ewmac_2_8_forecast,ewmac_2_8_forecast_abs
Date,,,,,,,,,,
1999-06-23,2956.00,49.851028,2947.000,NaN,NaN,2954.500000,2958.883420,-4.383420,-0.087930,0.087930
1999-06-24,2915.00,37.649691,2935.500,NaN,NaN,2927.837500,2943.503125,-15.665625,-0.416089,0.416089
1999-06-25,2911.50,32.252415,2913.250,NaN,NaN,2916.900826,2933.561716,-16.660889,-0.516578,0.516578
1999-07-20,3050.50,39.937902,3100.250,3119.93750,-19.68750,3084.706760,3107.522756,-22.815996,-0.571287,0.571287
1999-07-21,3067.00,38.545883,3058.750,3113.93750,-55.18750,3072.902253,3098.481804,-25.579551,-0.663613,0.663613
...,...,...,...,...,...,...,...,...,...,...
2020-09-21,10989.00,214.394412,10958.000,11146.71875,-188.71875,10992.896109,11166.952920,-174.056811,-0.811853,0.811853
2020-09-22,11149.50,212.347390,11069.250,11144.93750,-75.68750,11097.298703,11163.074493,-65.775790,-0.309756,0.309756
2020-09-23,10829.00,218.831406,10989.250,11117.68750,-128.43750,10918.432901,11088.835717,-170.402816,-0.778695,0.778695


In [ ]:
pd.DataFrame.diff

In [38]:
data_futures['NQZ20-CME']['mac_2_8'] = data_futures['NQZ20-CME']['ma_2'] - data_futures['NQZ20-CME']['ma_8']

In [42]:
data_futures['NQZ20-CME']['mac_2_8'].dropna().m

Date
1999-06-30     46.56250
1999-07-01     58.93750
1999-07-02     67.18750
1999-07-06     73.00000
1999-07-07     47.50000
                ...    
2020-10-09    170.37500
2020-10-12    363.62500
2020-10-13    483.34375
2020-10-14    328.71875
2020-10-15    168.12500
Name: mac_2_8, Length: 5367, dtype: float64

In [49]:
data_futures['NQZ20-CME'][' Last'].tail(757)

In [ ]:
pd.DataFrame.to_nk